In [3]:
from serpapi import GoogleSearch
import pandas as pd
import os, json
import mysql.connector
import pymysql

In [35]:
import plotly.express as px
import pandas as pd
import mysql.connector
import pymysql

cnx = mysql.connector.connect(
    host = "........."
    , port = 3306
    , user = 'jkober'
    , password = '.........!'
    , database = 'external_data'
    ,  auth_plugin='mysql_native_password'
)

# Create a cursor object to execute SQL queries
cursor = cnx.cursor()

table_query = """select * from external_data.vw_google_job_data_skills"""
cursor.execute(table_query)

# Fetch all the rows from the executed query
rows = cursor.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cursor.description]

# Create a DataFrame with the fetched data and columns
df = pd.DataFrame(rows, columns=columns)

# Display the DataFrame
print(df)


      skill requests  postings avg_salary  shown_salaries  \
0   Tableau       58       200      74857               7   
1  Power BI       47       200      61000               4   
2     Excel      105       200      72196              11   
3    Looker       23       200     105000               2   
4       SAS       18       200      31000               1   
5         R        8       200      57100               2   
6    Python       52       200      74925               8   
7       SQL      101       200      68203              12   
8     AI/ML       16       200      92667               6   

  senior_positions_offered senior_positions_total  
0                       20                     40  
1                       14                     40  
2                       21                     40  
3                        6                     40  
4                        1                     40  
5                        1                     40  
6                       1

In [4]:
params = {
  "api_key": "..................",
  "engine": "google_jobs",
  "google_domain": "google.com",
  "q": "data analyst",
  "hl": "en",
  "gl": "us",
  "location": "United States"
}

search = GoogleSearch(params)
results = search.get_dict()

In [5]:
# Generate a list of the first 10 pages of job listings
job_list = []

for offset in range(20):
    search.params_dict["start"] = offset * 10
    data = search.get_dict()
    for job_results in data['jobs_results']:
        job_dict = {
            "title": job_results["title"],
            "company_name": job_results["company_name"],
            "location": job_results["location"],
            "via": job_results["via"],
            "description": job_results["description"],
            "qualifications": job_results["job_highlights"][0]["items"],
            "link": job_results["related_links"][0]["link"],
            "extensions": job_results["detected_extensions"]
        }
        job_list.append(job_dict)

In [6]:
# turn job listings into a data frame, make sure variables are usable types
jobs = pd.DataFrame(job_list)
jobs['title'] = jobs['title'].astype(str)
jobs['company_name'] = jobs['company_name'].astype(str)
jobs['location'] = jobs['location'].astype(str)
jobs['via'] = jobs['via'].astype(str)
jobs['description'] = jobs['description'].astype(str)
jobs['qualifications'] = jobs['qualifications'].astype(str)
jobs['link'] = jobs['link'].astype(str)
jobs['extensions'] = jobs['extensions'].astype(str)
jobs.head(10)

,title,company_name,location,via,description,qualifications,link,extensions
0,Senior Data Analyst,"Blackbaud, Blackbaud",United States (+10 others),via Blackbaud Jobs,The Senior Data Analyst is a member of EVERFI’...,['At least 4 years of relevant work experience...,http://www.blackbaud.com/,"{'posted_at': '3 days ago', 'schedule_type': '..."
1,Data Analyst,ATC,United States,via LinkedIn,Job Title: Business Analyst\n\nWe are looking ...,['Bachelor’s degree in Business Administration...,https://www.google.com/search?sca_esv=57139995...,"{'posted_at': '1 day ago', 'schedule_type': 'F..."
2,Data Analyst II (Hybrid),One Gas,"Tulsa, OK",via ONE Gas Jobs,Position Description:\n\nJob Posting End Date:...,['You must have strong initiative and the abil...,http://www.onegas.com/,"{'posted_at': '2 days ago', 'schedule_type': '..."
3,Data Analyst,"LanceSoft, Inc.",Anywhere,via LinkedIn,Job Description\n\nCustomer Service Center has...,"[""Bachelor's degree in a quantitative field su...",http://www.lancesoft.com/,"{'posted_at': '1 day ago', 'schedule_type': 'C..."
4,Senior Data Analyst,Walmart,"Bentonville, AR",via Careers.walmart.com,Position Summary...\n\nWhat you'll do...\n\nAb...,['You have experience in People Analytics and ...,https://www.walmart.com/,"{'posted_at': '5 days ago', 'schedule_type': '..."
5,Data Analyst - Contract to Hire,Upwork,Anywhere,via Upwork,HIGH FREQUENCY TASKS….\n\nWork alongside CPO t...,['Work alongside CPO to ensure product goals a...,http://www.elance.com/,"{'posted_at': '4 hours ago', 'schedule_type': ..."
6,Marketing Data Analyst - Remote,GrowthAssistant,Anywhere,via Jobgether,"This a Full Remote job, the offer is available...",['2+ years experience with reporting and light...,https://www.google.com/search?sca_esv=57139995...,"{'posted_at': '1 day ago', 'schedule_type': 'F..."
7,Senior Data Analyst,Deloitte,"Wichita, KS",via Deloitte Jobs,Are you looking to make an impact by helping a...,['5 + years of experience with data quality an...,http://www.deloitte.com/,"{'posted_at': '2 days ago', 'schedule_type': '..."
8,Data Analyst with Snowflake experience,Akkodis,Anywhere,via LinkedIn,Akkodis is seeking a Data Analyst with Snowfla...,"['Cloud Technologies: Snowflake, AWS, Microsof...",http://www.akka-technologies.com/,"{'posted_at': '1 day ago', 'schedule_type': 'C..."
9,Operations Data Analyst and Modeler,GE,Anywhere,via GE Careers,Job Description Summary\nJoin our dynamic Oper...,"[""Bachelor's degree in Computer Science, Mathe...",http://www.ge.com/,"{'posted_at': '4 days ago', 'schedule_type': '..."


In [7]:
#Extract information from the Extensions variable
pattern = r"'posted_at':\s*'([^']+)'"
jobs['posted_at'] = jobs['extensions'].str.extract(pattern)
pattern = r"'schedule_type':\s*'([^']+)'"
jobs['schedule_type'] = jobs['extensions'].str.extract(pattern)
pattern = r"'work_from_home':\s*'([^']+)'"
jobs['work_from_home'] = jobs['extensions'].str.extract(pattern)
pattern = r"'salary':\s*'([^']+)'"
jobs['salary'] = jobs['extensions'].str.extract(pattern)

jobs.head(5)

,title,company_name,location,via,description,qualifications,link,extensions,posted_at,schedule_type,work_from_home,salary
0,Senior Data Analyst,"Blackbaud, Blackbaud",United States (+10 others),via Blackbaud Jobs,The Senior Data Analyst is a member of EVERFI’...,['At least 4 years of relevant work experience...,http://www.blackbaud.com/,"{'posted_at': '3 days ago', 'schedule_type': '...",3 days ago,Full-time,NaN,NaN
1,Data Analyst,ATC,United States,via LinkedIn,Job Title: Business Analyst\n\nWe are looking ...,['Bachelor’s degree in Business Administration...,https://www.google.com/search?sca_esv=57139995...,"{'posted_at': '1 day ago', 'schedule_type': 'F...",1 day ago,Full-time,NaN,NaN
2,Data Analyst II (Hybrid),One Gas,"Tulsa, OK",via ONE Gas Jobs,Position Description:\n\nJob Posting End Date:...,['You must have strong initiative and the abil...,http://www.onegas.com/,"{'posted_at': '2 days ago', 'schedule_type': '...",2 days ago,Full-time,NaN,NaN
3,Data Analyst,"LanceSoft, Inc.",Anywhere,via LinkedIn,Job Description\n\nCustomer Service Center has...,"[""Bachelor's degree in a quantitative field su...",http://www.lancesoft.com/,"{'posted_at': '1 day ago', 'schedule_type': 'C...",1 day ago,Contractor,NaN,60–70 an hour
4,Senior Data Analyst,Walmart,"Bentonville, AR",via Careers.walmart.com,Position Summary...\n\nWhat you'll do...\n\nAb...,['You have experience in People Analytics and ...,https://www.walmart.com/,"{'posted_at': '5 days ago', 'schedule_type': '...",5 days ago,Full-time,NaN,NaN


In [28]:
# Get the years of experience
jobs['years_of_experience'] = jobs['qualifications'].str.extract(r'(\S+)\s+years', expand=False)
jobs['years_of_experience'] = jobs['years_of_experience'].str.replace(r'\D', '', regex=True).str[-1]
jobs['years_of_experience'] = jobs['years_of_experience'].fillna('0').astype(int)
# Create Flags to show what skills are being asked for data analyst jobs
    # Top Tools for Data Analysis (Tableau, Power BI, Excel, Qlik, Domo, SAP, SPSS, Looker, SAS)
    # Top Languages for Data Analysis (python, r, sql, vba, javascript, c++)
    # Top Databases for Data Analysis (sql server, mysql, postgresql, mongodb)
    # Top Cloud Platforms for Data Analysis (snowflake, databricks, bigquery, oracle)
jobs['tableau'] =  jobs['qualifications'].str.lower().str.contains('tableau').astype(int)
jobs['power_bi'] =  jobs['qualifications'].str.lower().str.contains('power bi|powerbi').astype(int)
jobs['excel'] =  jobs['qualifications'].str.lower().str.contains('excel|microsoft').astype(int)
jobs['qlik'] =  jobs['qualifications'].str.lower().str.contains('qlik').astype(int)
jobs['domo'] =  jobs['qualifications'].str.lower().str.contains('domo').astype(int)
jobs['sap'] =  jobs['qualifications'].str.lower().str.contains('sap ').astype(int)
jobs['spss'] =  jobs['qualifications'].str.lower().str.contains('spss').astype(int)
jobs['looker'] =  jobs['qualifications'].str.lower().str.contains('looker').astype(int)
jobs['sas'] =  jobs['qualifications'].str.lower().str.contains('sas').astype(int)

jobs['python'] =  jobs['qualifications'].str.lower().str.contains('python').astype(int)
jobs['r'] =  jobs['qualifications'].str.lower().str.contains(' r |/r').astype(int)
jobs['sql'] =  jobs['qualifications'].str.lower().str.contains('sql').astype(int)
jobs['vba'] =  jobs['qualifications'].str.lower().str.contains('vba').astype(int)
jobs['javascript'] =  jobs['qualifications'].str.lower().str.contains('javascript').astype(int)

jobs['sql_server'] =  jobs['qualifications'].str.lower().str.contains('sql server').astype(int)
jobs['mysql'] =  jobs['qualifications'].str.lower().str.contains('mysql').astype(int)
jobs['postgresql'] =  jobs['qualifications'].str.lower().str.contains('postgresql').astype(int)
jobs['mongodb'] =  jobs['qualifications'].str.lower().str.contains('mongodb').astype(int)

jobs['snowflake'] =  jobs['qualifications'].str.lower().str.contains('snowflake').astype(int)
jobs['databricks'] =  jobs['qualifications'].str.lower().str.contains('databricks').astype(int)
jobs['bigquery'] =  jobs['qualifications'].str.lower().str.contains('bigquery').astype(int)
jobs['oracle'] =  jobs['qualifications'].str.lower().str.contains('oracle').astype(int)

# If it talks about work from home or remote in the description mark the Work From Home as true
jobs['remote'] =  jobs['description'].str.lower().str.contains('remote').astype(int)
# Gather flags for required education
jobs['bachelor_degree_required'] =  jobs['qualifications'].str.lower().str.contains('bachelor|bs/ba|ba/bs').astype(int)
jobs['master_degree_required'] =  jobs['qualifications'].str.lower().str.contains('master|ms/ma|ma/ms').astype(int)
jobs['requested_ai_experience'] = jobs['qualifications'].str.lower().str.contains('ai |machine learning|ml').astype(int)
jobs.head(10)

,title,company_name,location,via,description,qualifications,link,extensions,posted_at,schedule_type,...,postgresql,mongodb,snowflake,databricks,bigquery,oracle,remote,bachelor_degree_required,master_degree_required,requested_ai_experience
0,Senior Data Analyst,"Blackbaud, Blackbaud",United States (+10 others),via Blackbaud Jobs,The Senior Data Analyst is a member of EVERFI’...,['At least 4 years of relevant work experience...,http://www.blackbaud.com/,"{'posted_at': '3 days ago', 'schedule_type': '...",3 days ago,Full-time,...,0,0,0,0,0,0,1,0,0,0
1,Data Analyst,ATC,United States,via LinkedIn,Job Title: Business Analyst\n\nWe are looking ...,['Bachelor’s degree in Business Administration...,https://www.google.com/search?sca_esv=57139995...,"{'posted_at': '1 day ago', 'schedule_type': 'F...",1 day ago,Full-time,...,0,0,0,0,0,0,0,1,0,0
2,Data Analyst II (Hybrid),One Gas,"Tulsa, OK",via ONE Gas Jobs,Position Description:\n\nJob Posting End Date:...,['You must have strong initiative and the abil...,http://www.onegas.com/,"{'posted_at': '2 days ago', 'schedule_type': '...",2 days ago,Full-time,...,0,0,0,0,0,0,0,0,0,0
3,Data Analyst,"LanceSoft, Inc.",Anywhere,via LinkedIn,Job Description\n\nCustomer Service Center has...,"[""Bachelor's degree in a quantitative field su...",http://www.lancesoft.com/,"{'posted_at': '1 day ago', 'schedule_type': 'C...",1 day ago,Contractor,...,0,0,0,0,0,0,0,1,0,0
4,Senior Data Analyst,Walmart,"Bentonville, AR",via Careers.walmart.com,Position Summary...\n\nWhat you'll do...\n\nAb...,['You have experience in People Analytics and ...,https://www.walmart.com/,"{'posted_at': '5 days ago', 'schedule_type': '...",5 days ago,Full-time,...,0,0,0,0,0,0,0,1,1,0
5,Data Analyst - Contract to Hire,Upwork,Anywhere,via Upwork,HIGH FREQUENCY TASKS….\n\nWork alongside CPO t...,['Work alongside CPO to ensure product goals a...,http://www.elance.com/,"{'posted_at': '4 hours ago', 'schedule_type': ...",4 hours ago,Contractor and Temp work,...,0,0,0,0,0,0,0,0,0,0
6,Marketing Data Analyst - Remote,GrowthAssistant,Anywhere,via Jobgether,"This a Full Remote job, the offer is available...",['2+ years experience with reporting and light...,https://www.google.com/search?sca_esv=57139995...,"{'posted_at': '1 day ago', 'schedule_type': 'F...",1 day ago,Full-time,...,0,0,0,0,0,0,1,0,0,0
7,Senior Data Analyst,Deloitte,"Wichita, KS",via Deloitte Jobs,Are you looking to make an impact by helping a...,['5 + years of experience with data quality an...,http://www.deloitte.com/,"{'posted_at': '2 days ago', 'schedule_type': '...",2 days ago,Full-time,...,0,0,0,0,0,0,0,1,0,1
8,Data Analyst with Snowflake experience,Akkodis,Anywhere,via LinkedIn,Akkodis is seeking a Data Analyst with Snowfla...,"['Cloud Technologies: Snowflake, AWS, Microsof...",http://www.akka-technologies.com/,"{'posted_at': '1 day ago', 'schedule_type': 'C...",1 day ago,Contractor and Temp work,...,0,0,1,1,0,1,1,0,0,0
9,Operations Data Analyst and Modeler,GE,Anywhere,via GE Careers,Job Description Summary\nJoin our dynamic Oper...,"[""Bachelor's degree in Computer Science, Mathe...",http://www.ge.com/,"{'posted_at': '4 days ago', 'schedule_type': '...",4 days ago,Full-time,...,0,0,0,0,0,0,1,1,0,0


In [29]:
# Export Data to CSV
jobs.fillna('')
jobs.to_csv(r'job_postings.csv', index=False, header=True)


In [30]:

# Connect to MySQL server
cnx = mysql.connector.connect(
    host = "23.239.4.168"
    , port = 3306
    , user = 'jkober'
    , password = '4N4CrFHevPzpdt!'
    , database = 'external_data'
    ,  auth_plugin='mysql_native_password'
)

# Create a cursor object to execute SQL queries
cursor = cnx.cursor()

# Query to drop table
drop_table_query = """
    drop table external_data.google_job_data
"""
cursor.execute(drop_table_query)

# Query to create table
create_table_query = """
    create table external_data.google_job_data (
        title varchar(100)	
        , company_name varchar(100)
        , location	varchar(50)
        , link varchar(200)
        , posted_at	varchar(50)
        , schedule_type	varchar(50)
        , work_from_home varchar(50)	
        , salary varchar(50)
        , tableau int
        , power_bi int
        , excel int
        , qlik int
        , domo int
        , sap int
        , spss int
        , looker int
        , sas int
        , python int
        , r int
        , sql_skills int
        , vba int
        , javascript int
        , sql_server int
        , mysql int
        , postgresql int
        , mongodb int
        , snowflake int
        , databricks int
        , bigquery int
        , oracle int
        , remote int
        , years_of_experience int
        , bachelor_degree_required int
        , master_degree_required int
        , requested_ai_experience int
    )
    """
cursor.execute(create_table_query)



In [31]:
# Insert Data into the table for jobs
for i in range(len(jobs)):
    sql_jobs = "INSERT INTO external_data.google_job_data (title, company_name, location, link, posted_at, schedule_type, work_from_home, salary, tableau, power_bi, excel, qlik, domo, sap, spss, looker, sas, python, r, sql_skills, vba, javascript, sql_server, mysql, postgresql, mongodb, snowflake, databricks, bigquery, oracle, remote, years_of_experience, bachelor_degree_required, master_degree_required, requested_ai_experience) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values_jobs = (
        jobs['title'].iloc[i]
        , jobs['company_name'].iloc[i]
        , jobs['location'].iloc[i]
        , jobs['link'].iloc[i]
        , jobs['posted_at'].iloc[i]
        , jobs['schedule_type'].iloc[i]
        , jobs['work_from_home'].iloc[i]
        , jobs['salary'].iloc[i]
        , int(jobs['tableau'].iloc[i])
        , int(jobs['power_bi'].iloc[i])
        , int(jobs['excel'].iloc[i])
        , int(jobs['qlik'].iloc[i])
        , int(jobs['domo'].iloc[i])
        , int(jobs['sap'].iloc[i])
        , int(jobs['spss'].iloc[i])
        , int(jobs['looker'].iloc[i])
        , int(jobs['sas'].iloc[i])
        , int(jobs['python'].iloc[i])
        , int(jobs['r'].iloc[i])
        , int(jobs['sql'].iloc[i])
        , int(jobs['vba'].iloc[i])
        , int(jobs['javascript'].iloc[i])
        , int(jobs['sql_server'].iloc[i])
        , int(jobs['mysql'].iloc[i])
        , int(jobs['postgresql'].iloc[i])
        , int(jobs['mongodb'].iloc[i])
        , int(jobs['snowflake'].iloc[i])
        , int(jobs['databricks'].iloc[i])
        , int(jobs['bigquery'].iloc[i])
        , int(jobs['oracle'].iloc[i])
        , int(jobs['remote'].iloc[i])
        , int(jobs['years_of_experience'].iloc[i])
        , int(jobs['bachelor_degree_required'].iloc[i])
        , int(jobs['master_degree_required'].iloc[i])
        , int(jobs['requested_ai_experience'].iloc[i]))
        
    # Execute the INSERT statement
    cursor.execute(sql_jobs, values_jobs)

# Close the connection
cnx.commit()
cnx.close()